In [1]:
import sys
sys.path.append("../")

import argparse
import datetime
import json
import os
import re
from time import sleep

import numpy as np
import pandas as pd
import requests

from vook_db_v7.config import MAX_PAGE, REQ_URL, WANT_ITEMS, req_params

# from logzero import logger

In [2]:
# apiコールした結果からdataframeを出力する関数を定義
def DataFrame_maker(keyword, platform_id, knowledge_id, size_id):
    cnt = 1
    df = pd.DataFrame(columns=WANT_ITEMS)
    req_params["page"] = cnt
    req_params["keyword"] = keyword
    while True:
        req_params["page"] = cnt
        res = requests.get(REQ_URL, req_params)
        res_code = res.status_code
        res = json.loads(res.text)
        if res_code != 200:
            print(
                f"""
            ErrorCode -> {res_code}\n
            Error -> {res['error']}\n
            Page -> {cnt}"""
            )
        else:
            if res["hits"] == 0:
                print("返ってきた商品数の数が0なので、ループ終了")
                break
            tmp_df = pd.DataFrame(res["Items"])[WANT_ITEMS]
            df = pd.concat([df, tmp_df], ignore_index=True)
        if cnt == MAX_PAGE:
            print("MAX PAGEに到達したので、ループ終了")
            break
        # logger.info(f"{cnt} end!")
        cnt += 1
        # リクエスト制限回避
        sleep(1)

        print("Finished!!")
    df["platform_id"] = platform_id
    df["knowledge_id"] = knowledge_id
    df["size_id"] = size_id
    df_main = df.rename(
        columns={"itemName": "name", "itemPrice": "price", "itemUrl": "url"}
    )
    df_main = df_main.reindex(
        columns=["id", "name", "url", "price", "knowledge_id", "platform_id", "size_id"]
    )
    print("price type before:", df_main["price"].dtype)
    df_main["price"] = df_main["price"].astype(int)
    print("price type after:", df_main["price"].dtype)
    run_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    df_main["created_at"] = run_time
    df_main["updated_at"] = run_time
    return df_main

In [16]:
platform_id = 1 #楽天
size_id = 999

'''
#<Knowledge:0x00007f7a0bb944c0
  id: 5,
  name: "66前期",
  age: 1974,
  brand_id: 1,
  item_id: 1,
  line_id: 1,

  #<Knowledge:0x00007f7a100d30e0
  id: 19,
  name: "66後期",
  age: 1977,
  brand_id: 1,
  item_id: 1,
  line_id: 1,

  #<Knowledge:0x00007f7a100d35e0
  id: 13,
  name: "ギャラ入り",
  age: 1955,
  brand_id: 1,
  item_id: 1,
  line_id: 2,

#<Brand:0x00007f7a0b534918
  id: 1,
  name: "Levi's",

  id: 2,
  name: "Wrangler",

#<Line:0x00007f7a10081d80
  id: 1,
  name: "501",
  brand_id: 1,
  item_id: 1,

  #<Line:0x00007f7a100d12e0
  id: 2,
  name: "501XX",
  brand_id: 1,
  item_id: 1,
'''

#ブランドIdとブランドnameを対応させてたテーブルを作成する
csv_sample = pd.DataFrame()

df_brand_name = pd.DataFrame(data={'brand':["Levi's","Wrangler"]})
df_brandid = pd.DataFrame(data={'brand_id':[1,2]})
df_brand = pd.concat([df_brand_name, df_brandid], axis=1)
df_brand

,brand,brand_id
0,Levi's,1
1,Wrangler,2


In [12]:
df_line_name = pd.DataFrame(data={'line_name':["501","501XX"]})
df_line_id = pd.DataFrame(data={'line_id':[1,2]})
df_line = pd.concat([df_line_name, df_line_id], axis=1)
df_line

,line_name,line_id
0,501,1
1,501XX,2


In [54]:
data1

{'line_id': [5],
 'knowledge_name': ['66前期'],
 'brand_name': ["Levi's"],
 'line_name': ['501']}

In [71]:
data1 = {"knowledge_id":[5],
         "knowledge_name":["66前期"],
         "brand_name": ["Levi's"],
         "line_name":["501"]}
data1 = pd.DataFrame(data1)
data2 = {"knowledge_id":[19],
         "knowledge_name":["66後期"],
         "brand_name": ["Levi's"],
         "line_name":["501"]}
data2 = pd.DataFrame(data2)

data = pd.concat([data1,data2], axis=0, ignore_index=True)
data

,knowledge_id,knowledge_name,brand_name,line_name
0,5,66前期,Levi's,501
1,19,66後期,Levi's,501


In [72]:
n_bulk = len(data)
n_bulk

df_bulk = pd.DataFrame()

for n in np.arange(n_bulk):
    brand_name = data.brand_name[n]
    line_name = data.line_name[n]
    knowledge_name = data.knowledge_name[n]
    query = f"{brand_name} {line_name} {knowledge_name} 中古"
    knowledge_id = data.knowledge_id[n]
    print("検索キーワード:[" + query + "]", "knowledge_id:", knowledge_id)
    output = DataFrame_maker(query, platform_id, knowledge_id, size_id)
    df_bulk = pd.concat([df_bulk, output], ignore_index=True)

検索キーワード:[Levi's 501 66前期 中古] knowledge_id: 5
Finished!!
Finished!!
Finished!!
Finished!!
返ってきた商品数の数が0なので、ループ終了
price type before: object
price type after: int64
検索キーワード:[Levi's 501 66後期 中古] knowledge_id: 19
Finished!!
Finished!!
Finished!!
返ってきた商品数の数が0なので、ループ終了
price type before: object
price type after: int64


In [73]:
df_bulk

,id,name,url,price,knowledge_id,platform_id,size_id,created_at,updated_at
0,NaN,【中古】未使用品 リーバイス Levi's 501 66前期 70's ヴィンテージ デッド...,https://hb.afl.rakuten.co.jp/hgc/g00qosae.brn1...,701000,5,1,999,2023-11-11 16:23:59.097625,2023-11-11 16:23:59.097625
1,NaN,【中古】未使用品 リーバイス Levi's 501 66前期 70's ヴィンテージ デッド...,https://hb.afl.rakuten.co.jp/hgc/g00rsgoe.brn1...,701000,5,1,999,2023-11-11 16:23:59.097625,2023-11-11 16:23:59.097625
2,NaN,Levi's リーバイス デニムパンツ ヴィンテージ 501 66前期 ビックE トップボタ...,https://hb.afl.rakuten.co.jp/hgc/g00rc6oe.brn1...,395780,5,1,999,2023-11-11 16:23:59.097625,2023-11-11 16:23:59.097625
3,NaN,リーバイス 70s 501 66前期 BIGE ストレート デニムパンツ W31 Levi&...,https://hb.afl.rakuten.co.jp/hgc/g00rf2ie.brn1...,327800,5,1,999,2023-11-11 16:23:59.097625,2023-11-11 16:23:59.097625
4,NaN,Levi's(リーバイス) サイズ:W34 L31 70s Vintage 501 66前期...,https://hb.afl.rakuten.co.jp/hgc/g00s6rie.brn1...,325050,5,1,999,2023-11-11 16:23:59.097625,2023-11-11 16:23:59.097625
...,...,...,...,...,...,...,...,...,...
187,NaN,【中古】Levi’s◆ボトム/31/デニム/IDG/無地/リーバイス/66後期/501/ダメ...,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,11990,19,1,999,2023-11-11 16:24:02.585020,2023-11-11 16:24:02.585020
188,NaN,リーバイス 501 赤耳 66後期 1980年代 Levi's W28.5 L25.5【古着...,https://hb.afl.rakuten.co.jp/hgc/g00qecve.brn1...,8800,19,1,999,2023-11-11 16:24:02.585020,2023-11-11 16:24:02.585020
189,NaN,リーバイス 80s 501 66後期 ボタンフライ デニムショートパンツ w30 Levi'...,https://hb.afl.rakuten.co.jp/hgc/g00rf2ie.brn1...,7573,19,1,999,2023-11-11 16:24:02.585020,2023-11-11 16:24:02.585020
190,NaN,【中古】Levi’s◆70s/66後期/刻印6/カットオフ/ショートパンツ/30/デニム/I...,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,6490,19,1,999,2023-11-11 16:24:02.585020,2023-11-11 16:24:02.585020
